# Clasificación de Text
Créditos: https://keras.rstudio.com/articles/tutorial_basic_text_classification.html


El siguiente tutorial consisten en clasificar reseñas de peliculas como  positivas o negativas utilizando el texto de la reseña. Este es un ejemplo de clasificacion binaria, un tipo de problema de aprendizaje automatico importante y apliamente aplicable.

Se utilizara el conjunto de datos IMDB que contiene el texto de 50,000 reseñas de peliculas de la base de datos de internet. Estas se dividen en 25,000 reseñas para el entrenamiento y 25,000 reseñas para la validacion. Los conjuntos de entrenamiento y validacion son equilibrados, lo que significa que contiene el mismo numero de reseñas positivas y negativas.


In [4]:
#Librerias necesarias

library(keras)#necesario usar la funcion install_kereas() para instalar kerear, si no funcinoa intalar r studio en conda
library(dplyr)
library(ggplot2)
library(purrr)
library(tensorflow)

### Descargar conjuntos de datos IMDB

El conjunto de datos IMDP vine con la libreria keras. Ya se ha procesados de manera tal que las reseñas (secuencias de palabras) se han convertido en secuencias de enteros, donde cada entero representa un palabra especifica en un diccionario.

El siguiente codigo descarga eñ conjunto de datos IMDB (o usa una copia en cache si ya se ha descargado).

In [5]:
imdb<-dataset_imdb(num_words = 10000)

c(train_data, train_labels) %<-% imdb$train
c(test_data, test_labels) %<-% imdb$test

El argumento num_words = 10000 mantiene las 10,000 palabras mas frecuentes en los datos de entrenamiento. Las plabras raras se descartan para mantener manejable el tamaño de los datos.

Convenientemente, el conjunto de datos viene con un indice que asigna palabras a enteros, que deben descargarse por separado.

In [8]:
word_index<-dataset_imdb_word_index()
head(word_index,3)

$fawn
[1] 34701

$tsukino
[1] 52006

$nunnery
[1] 52007

### Exploracion de datos 

Tomemos un poco de tiempo para entender el formato de los datos. El conjunto de datos viene preprocesado: cada ejemplo es un conjunto de enteros que representan las palabras de la reseña de la pelicula. Cada etiqueta es un valor entero de 0 o 1, donde 0 es una reseña negativa y 1 es una reseña positiva.

In [10]:
paste0("Training entries: ", length(train_data), ", labels: ", length(train_labels))

[1] "Training entries: 25000, labels: 25000"

Los textos de la reseñas se han tranformado en numeros enteros, donde cada numero entero representa una palabra especifica en un diccionario. Asi es como se ve la primera reseña


In [11]:
train_data[[1]]

[1]    1   14   22   16   43  530  973 1622 1385   65  458 4468   66 3941    4
 [16]  173   36  256    5   25  100   43  838  112   50  670    2    9   35  480
 [31]  284    5  150    4  172  112  167    2  336  385   39    4  172 4536 1111
 [46]   17  546   38   13  447    4  192   50   16    6  147 2025   19   14   22
 [61]    4 1920 4613  469    4   22   71   87   12   16   43  530   38   76   15
 [76]   13 1247    4   22   17  515   17   12   16  626   18    2    5   62  386
 [91]   12    8  316    8  106    5    4 2223 5244   16  480   66 3785   33    4
[106]  130   12   16   38  619    5   25  124   51   36  135   48   25 1415   33
[121]    6   22   12  215   28   77   52    5   14  407   16   82    2    8    4
[136]  107  117 5952   15  256    4    2    7 3766    5  723   36   71   43  530
[151]  476   26  400  317   46    7    4    2 1029   13  104   88    4  381   15
[166]  297   98   32 2071   56   26  141    6  194 7486   18    4  226   22   21
[181]  134  476   26  480    5  144   30 5535   18   51   36   28  224   92   25
[196]  104    4  226   65   16   38 1334   88   12   16  283    5   16 4472  113
[211]  103   32   15   16 5345   19  178   32

Las reseñas de peliculas pueden tener diferentes longitudes. El siguiente codigo muestra el numero de palabras en la primera y segunda reseña. Dado que las entradas a una red neuronal deben tener la misma lingitud, se tiene que resolver esto mas adelante.

In [12]:
sapply(train_data[1:2],length)

[1] 218 189

### Convertir enteros a palabras

Puede se util saber como convertir enteros de nuevo a texto. Ya se sabe que word_index tiene el catalogo de palabra. Si se crea un data.frame a partir de el, se puede usar en ambas direcciones.

In [13]:
word_index_df<-data.frame(
  word = names(word_index),
  idx = unlist(word_index,use.names = FALSE),
  stringsAsFactors = FALSE
)

Los primeros índices son reservados

In [15]:
word_index_df <- word_index_df%>%
  mutate(idx = idx + 3)%>%
  add_row(word = "<PAD>", idx = 0)%>%
  add_row(word = "<START>", idx = 1)%>%
  add_row(word = "<UNK>", idx = 2)%>%
  add_row(word = "<UNUSED>", idx = 3)%>%
  arrange(idx)

decode_review <- function(text){
  paste(map(text, function(number) word_index_df %>%
              filter(idx == number) %>%
              select(word) %>% 
              pull()),
        collapse = " ")
}

Ahora se puede usar la funcion decode_view para mostrar el texto para la primera reseña

In [16]:
decode_review(train_data[[1]])

[1] "<START> this film was just brilliant casting location scenery story direction everyone's really suited the part they played and you could just imagine being there robert <UNK> is an amazing actor and now the same being director <UNK> father came from the same scottish island as myself so i loved the fact there was a real connection with this film the witty remarks throughout the film were great it was just brilliant so much that i bought the film as soon as it was released for <UNK> and would recommend it to everyone to watch and the fly fishing was amazing really cried at the end it was so sad and you know what they say if you cry at a film it must have been good and this definitely was also <UNK> to the two little boy's that played the <UNK> of norman and paul they were just brilliant children are often left out of the <UNK> list i think because the stars that play them all grown up are such a big profile for the whole film but these children are amazing and should be praised for what they have done don't you think the whole story was so lovely because it was true and was someone's life after all that was shared with us all"

### Prepacion de datos

Las reseña, los arreglos de enteros, deben transformarse a tensores antes de alimentar la red neuronal. Esta convension se puede realizar de dos maneras. Codificando en caliente las matrices para transformarlas en vector de 0's y 1's. Por ejemplo, se secuencia [3,5] se tranformaria en un vector de dimension 10,000 que son todos ceros, excepto los indices 3 y 5, que son unos. 

Entonces, se hace esta la primera capa de la red, una capa densa, que pueda manejar datos de vectores de punto flotante. Sin embargo, este enfoque requiere mucha memoria y requiere un matriz de tamaño num_words * num_reviews(reseñas). De manera alternativa, se puede rellenar los areglos de enteros de maenra que tengan la misma longitud, y luego crear un tensor de tamaño num_examples * max_length. Se puede usar una cada de inclusion capaz de manejar esta forma como la primera capa en la red.

Dedibo a que las reseñas deben dener la misma longitud, se utilizara la funcion pad_sequences para estanzarizar las longitudes

In [19]:
train_data<-pad_sequences(
  train_data,
  value = word_index_df%>%filter(word == "<PAD>")%>%select(idx)%>%pull(),
  padding =  "post",
  maxlen = 256
)

### Construccion del modelo

La red neuronal se crea apilando capas, esto requiere dos decisiones arquitectonicas principales: ¿cuántas capas usar en el modelo? ¿cuántas unidades ocultas usar para cada capa?
 
En este emjemplo, los datos de entrada consisten en una matriz de indices de palabras. Las etiquetas para predecir si son 0 o 1. 

Construyamos un modelo para este problema.
La forma de entrada es un contenceo de palabras usado para la reseña de la pelicual (10,000 palabras).

In [20]:
vocab_size<-10000

model<-keras_model_sequential()

model%>%
  layer_embedding(input_dim = vocab_size, output_dim = 16)%>%
  layer_global_average_pooling_1d()%>%
  layer_dense(units = 16, activation = "relu")%>%
  layer_dense(units = 1, activation = "sigmoid")

model%>%summary()

Model: "sequential"
________________________________________________________________________________
Layer (type)                        Output Shape                    Param #     
embedding (Embedding)               (None, None, 16)                160000      
________________________________________________________________________________
global_average_pooling1d (GlobalAve (None, 16)                      0           
________________________________________________________________________________
dense (Dense)                       (None, 16)                      272         
________________________________________________________________________________
dense_1 (Dense)                     (None, 1)                       17          
Total params: 160,289
Trainable params: 160,289
Non-trainable params: 0
________________________________________________________________________________


Las capas se apilan de manera secuencial para construir el clasificador:

La primer capa es una cama embebida. Esta capa toma el vocabulario codificado con enteros y busca el vector de incrustacion para ca indice de palabras. Estos vectores se aprender a medida que el modelo entrena.
Los vectores agregan una dimension a la matriz de salida. Las dimensiones resultanes son: (tach, sequence, embedding).


In [22]:
model %>% compile(
  optimizer = 'adam',
  loss = 'binary_crossentropy',
  metrics = list('accuracy')
)

### Crear un conjunto de validación

In [23]:
x_val <- train_data[1:10000, ]
partial_x_train <- train_data[10001:nrow(train_data), ]

y_val <- train_labels[1:10000]
partial_y_train <- train_labels[10001:length(train_labels)]

### Entrenamos el modelo

In [24]:
history <- model %>% fit(
  partial_x_train,
  partial_y_train,
  epochs = 40,
  batch_size = 512,
  validation_data = list(x_val, y_val),
  verbose=1
)

### Evaluar el modelo

In [ ]:
results <- model %>% evaluate(test_data, test_labels)
results


predictions<-model%>%predict(test_data)

cbind(round(predictions,5),test_labels)
